# Transform Payments Data
1. Extract Date and Time from payment_timestamp and create new columns payment_date and payment_time
2. Map payment_status to contain descriptive values  
(1-Success, 2-Pending, 3-Cancelled, 4-Failed)
3. Write Transformed data to the Silver Schema

In [0]:
payments_df = spark.table("gizmobox.bronze.payments")
display(payments_df)

### 1. Extract Date and Time from payment_timestamp and create new columns payment_date and payment_time

In [0]:
from pyspark.sql.functions import *
df_extracted_payments = payments_df.select('payment_id',
                                  'order_id',
                                  date_format('payment_timestamp', 'yyyy-MM-dd').cast('date').alias('payment_date'),
                                  date_format('payment_timestamp', 'HH-mm-ss').alias('payment_time'),
                                  'payment_status',
                                  'payment_method'
                                  )
display(df_extracted_payments)

### 2. Map payment_status to contain descriptive values (1-Success, 2-Pending, 3-Cancelled, 4-Failed)


In [0]:
df_mapped_payments = df_extracted_payments.select('payment_id', 'order_id', 'payment_date', 'payment_time', 
                             when(df_extracted_payments.payment_status ==1, 'Success') \
                             .when(df_extracted_payments.payment_status ==2, 'Pending') \
                             .when(df_extracted_payments.payment_status ==3, 'Cancelled') \
                             .when(df_extracted_payments.payment_status ==4, 'Failed').alias('payment_status'),
                             'payment_method'
                             )
display(df_mapped_payments)

### 3. Write Transformed data to the Silver Schema

In [0]:
df_mapped_payments.writeTo('gizmobox.silver.py_payments').createOrReplace()

In [0]:
%sql
select * from gizmobox.silver.py_payments;